In [265]:
import numpy as np
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.stats import boxcox
from scipy import sparse
from sklearn.model_selection import StratifiedKFold
from itertools import product
from sklearn import preprocessing

from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.metrics import log_loss

from subprocess import check_output
from scipy.stats import pearsonr

from os import listdir
from os.path import isfile, join
from scipy.stats import rankdata
import zipfile

pd.set_option('display.max_rows',100)

import seaborn as sns
import matplotlib.pyplot as plt

In [19]:
def merge_all_result(path,key,df):
    file_list = []
    file_path_list=[]
    try:
        for file in listdir(path):

            if file.find('.csv')==-1:
                continue

            print(file)    
            file_list.append(file)
            file_path = join(path,file)
            file_path_list.append(file_path)
            temp = pd.read_csv(file_path)
            #predict_list.append(temp[LABELS].values)
            if df.empty == True:
                print("empty")
                df = df.append(temp)
            else:
                print("in")
                df = df.merge(temp,on=key,how='left')
    except BaseException as e:
        print("exception")
        print(e)
    finally:
        return df

In [208]:
def confution_matrix(oof, target, base):
    pred = oof.copy()
    pred = pd.Series(pred)
    pred[pred<=base] = 0
    pred[pred>base] = 1
    tp = np.sum(np.logical_and(pred==1, target==1))
    tn = np.sum(np.logical_and(pred==0, target==0))
    fp = np.sum(np.logical_and(pred==1, target==0))
    fn = np.sum(np.logical_and(pred==0, target==1))  
    return tp, fn, fp, tn

# False인데 True라고 한 비율
def get_score(oof, target, base):
    tp, fn, fp, tn = confution_matrix(oof, target, base)
    print("Base: ", base)
    print(tp-fp, tp, fn, fp, tn) 
    precision = tp / (tp+fp)
    recall = tp/(tp+fn)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F Score: ", 2*(precision*recall)/(precision+recall))
    print("FDR: ", fp/(tp+fp))
    
def get_best_ratio(oof, target, base):
    tp, fn, fp, tn = confution_matrix(oof, target, base)
    return tp-fp, 

In [20]:
train_outlier_path = 'ensemble/train_oof/'
test_outlier_path = 'ensemble/test_oof/'

In [21]:
train = pd.read_csv('input/train.csv')
train['outlier'] = 0
train.loc[train['target']<-30,'outlier'] = 1

train oof

In [280]:
data = pd.DataFrame()
data = merge_all_result(train_outlier_path,'card_id',data)
data = data.merge(train[['card_id','outlier','target']], on='card_id', how='left')
data.columns = ['card_id', 'yh_target', 'train_oof_cat_auc_real','train_oof_lgb_auc_real', 'train_oof_lgb_log_loss_real',
       'train_yeonmin_rank_kfold_stacking_0', 'train_yeonmin_rank_kfold_stacking_1',
       'train_yeonmin_rank_kfold_stacking_2', 'train_yeonmin_rank_kfold_stacking_3', 'train_yeonmin_rank_kfold_stacking_4',
       'train_yeonmin_rank_kfold_stacking2_0', 'train_yeonmin_rank_kfold_stacking2_1',
       'train_yeonmin_rank_kfold_stacking2_2', 'train_yeonmin_rank_kfold_stacking2_3', 'train_yeonmin_rank_kfold_stacking2_4',
       'train_yeonmin_rank_sfold_stacking_0', 'train_yeonmin_rank_sfold_stacking_1', 'train_yeonmin_rank_sfold_stacking_2', 
       'train_yeonmin_rank_sfold_stacking_3', 'train_yeonmin_rank_sfold_stacking_4', 
       'train_yeonmin_rank_sfold_stacking2_0', 'train_yeonmin_rank_sfold_stacking2_1', 'train_yeonmin_rank_sfold_stacking2_2', 
       'train_yeonmin_rank_sfold_stacking2_3', 'train_yeonmin_rank_sfold_stacking2_4',
        'outlier','target']

train_oof_augmented_2500_auc_0.9057314966569757_logloss__function log_loss at 0x7f9ab3285bf8__v1.csv
empty
train_oof_cat_auc_real.csv
in
train_oof_lgb_auc_real.csv
in
train_oof_lgb_log_loss_real.csv
in
train_yeonmin_rank_kfold_stacking.csv
in
train_yeonmin_rank_kfold_stacking2.csv
in
train_yeonmin_rank_sfold_stacking.csv
in
train_yeonmin_rank_sfold_stacking2.csv
in


In [274]:
col_list = list(data.columns)
col_list.remove('card_id')
col_list.remove('outlier')
col_list.remove('target')

In [275]:
col_base_ratio_dict = {}
for col in col_list:
    print(col)
    max_diff_tp_fp = -100
    max_diff_tp_fp_ratio = 0
    max_tp = 0
    max_fp = 0
    for base in np.arange(0.3,0.8,0.01):
        tp, fn, fp, tn = confution_matrix(data[col], data['outlier'], base)
        diff = tp-fp
        if diff >= max_diff_tp_fp:
            max_diff_tp_fp = diff
            max_diff_tp_fp_ratio = base
            max_tp = tp
            max_fp = fp
            
    print(max_diff_tp_fp, max_diff_tp_fp_ratio, max_tp, max_fp)
    col_base_ratio_dict[col] = max_diff_tp_fp_ratio
    data.loc[data[col]>=max_diff_tp_fp_ratio, col] = 1
    data.loc[data[col]!=1, col] = 0
    print("")
    print("")

yh_target
4 0.5300000000000002 4 0


train_oof_cat_auc_real
1 0.47000000000000014 2 1


train_oof_lgb_auc_real
8 0.5100000000000002 13 5


train_oof_lgb_log_loss_real
11 0.5300000000000002 12 1


train_yeonmin_rank_kfold_stacking_0
5 0.5200000000000002 10 5


train_yeonmin_rank_kfold_stacking_1
6 0.46000000000000013 40 34


train_yeonmin_rank_kfold_stacking_2
6 0.5500000000000003 11 5


train_yeonmin_rank_kfold_stacking_3
3 0.5100000000000002 18 15


train_yeonmin_rank_kfold_stacking_4
5 0.5200000000000002 9 4


train_yeonmin_rank_kfold_stacking2_0
12 0.47000000000000014 32 20


train_yeonmin_rank_kfold_stacking2_1
8 0.5500000000000003 14 6


train_yeonmin_rank_kfold_stacking2_2
13 0.49000000000000016 29 16


train_yeonmin_rank_kfold_stacking2_3
6 0.5400000000000003 9 3


train_yeonmin_rank_kfold_stacking2_4
5 0.5000000000000002 12 7


train_yeonmin_rank_sfold_stacking_0
11 0.49000000000000016 27 16


train_yeonmin_rank_sfold_stacking_1
12 0.5100000000000002 30 18


train_yeonmin_rank_

In [276]:
data['voting'] = data[col_list].mean(axis=1)

In [277]:
data.sort_values('voting',ascending=False)

,card_id,yh_target,train_oof_cat_auc_real,train_oof_lgb_auc_real,train_oof_lgb_log_loss_real,train_yeonmin_rank_kfold_stacking_0,train_yeonmin_rank_kfold_stacking_1,train_yeonmin_rank_kfold_stacking_2,train_yeonmin_rank_kfold_stacking_3,train_yeonmin_rank_kfold_stacking_4,...,train_yeonmin_rank_sfold_stacking_3,train_yeonmin_rank_sfold_stacking_4,train_yeonmin_rank_sfold_stacking2_0,train_yeonmin_rank_sfold_stacking2_1,train_yeonmin_rank_sfold_stacking2_2,train_yeonmin_rank_sfold_stacking2_3,train_yeonmin_rank_sfold_stacking2_4,outlier,target,voting
103013,C_ID_b8806d02d1,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,-33.219281,0.958333
25347,C_ID_0b358310f5,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1,-33.219281,0.875000
70485,C_ID_26255f7db0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1,-33.219281,0.833333
25880,C_ID_ec83c5deb0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1,-33.219281,0.833333
45032,C_ID_f467f10386,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1,-33.219281,0.791667
60526,C_ID_298998e8b2,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1,-33.219281,0.708333
179088,C_ID_ed2b2ab4ea,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,-33.219281,0.708333
21057,C_ID_26e9fcf59c,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0,-5.870283,0.708333
171585,C_ID_ed579b74fc,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1,-33.219281,0.625000
15933,C_ID_b7d3533686,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0.204692,0.625000


In [278]:
data.sort_values('voting',ascending=False).to_csv('voting.csv',index=False)

In [255]:
data.columns

Index(['card_id', 'yh_target', 'train_oof_cat_auc_real',
       'train_oof_lgb_auc_real', 'train_oof_lgb_log_loss_real',
       'train_yeonmin_rank_kfold_stacking_0',
       'train_yeonmin_rank_kfold_stacking_1',
       'train_yeonmin_rank_kfold_stacking_2',
       'train_yeonmin_rank_kfold_stacking_3',
       'train_yeonmin_rank_kfold_stacking_4',
       'train_yeonmin_rank_kfold_stacking2_0',
       'train_yeonmin_rank_kfold_stacking2_1',
       'train_yeonmin_rank_kfold_stacking2_2',
       'train_yeonmin_rank_kfold_stacking2_3',
       'train_yeonmin_rank_kfold_stacking2_4',
       'train_yeonmin_rank_sfold_stacking_0',
       'train_yeonmin_rank_sfold_stacking_1',
       'train_yeonmin_rank_sfold_stacking_2',
       'train_yeonmin_rank_sfold_stacking_3',
       'train_yeonmin_rank_sfold_stacking_4',
       'train_yeonmin_rank_sfold_stacking2_0',
       'train_yeonmin_rank_sfold_stacking2_1',
       'train_yeonmin_rank_sfold_stacking2_2',
       'train_yeonmin_rank_sfold_stacking2_

In [269]:
rank_data = data[col_list].rank(ascending=False)
rank_data = pd.concat([data[['card_id']],rank_data,data[['outlier','target']]],axis=1)
rank_data['outlier_rank'] = rank_data[col_list].mean(axis=1)

In [270]:
rank_data.sort_values('outlier_rank')

,card_id,yh_target,train_oof_cat_auc_real,train_oof_lgb_auc_real,train_oof_lgb_log_loss_real,train_yeonmin_rank_kfold_stacking_0,train_yeonmin_rank_kfold_stacking_1,train_yeonmin_rank_kfold_stacking_2,train_yeonmin_rank_kfold_stacking_3,train_yeonmin_rank_kfold_stacking_4,...,train_yeonmin_rank_sfold_stacking_3,train_yeonmin_rank_sfold_stacking_4,train_yeonmin_rank_sfold_stacking2_0,train_yeonmin_rank_sfold_stacking2_1,train_yeonmin_rank_sfold_stacking2_2,train_yeonmin_rank_sfold_stacking2_3,train_yeonmin_rank_sfold_stacking2_4,outlier,target,outlier_rank
103013,C_ID_b8806d02d1,1.0,21.0,17.0,5.0,3.0,4.0,13.0,3.0,3.0,...,13.0,7.0,6.0,1.0,12.0,17.0,8.0,1,-33.219281,7.041667
70485,C_ID_26255f7db0,9.0,4.0,2.0,3.0,1.0,1.0,6.0,25.0,1.0,...,4.0,2.0,1.0,90.0,5.0,5.0,2.0,1,-33.219281,10.916667
25880,C_ID_ec83c5deb0,24.0,2.0,1.0,2.0,2.0,19.0,41.0,4.0,2.0,...,9.0,1.0,5.0,10.0,65.0,16.0,6.0,1,-33.219281,13.291667
45032,C_ID_f467f10386,25.0,30.0,4.0,6.0,8.0,2.0,4.0,2.0,40.0,...,5.0,3.0,4.0,5.0,104.0,6.0,7.0,1,-33.219281,14.666667
21057,C_ID_26e9fcf59c,45.0,19.0,3.0,1.0,4.0,8.0,1.0,1.0,51.0,...,2.0,5.0,30.0,2.0,22.0,13.0,5.0,0,-5.870283,16.333333
15933,C_ID_b7d3533686,34.0,5.0,54.0,29.0,57.0,47.0,11.0,23.0,41.0,...,6.0,6.0,2.0,9.0,3.0,2.0,1.0,0,0.204692,21.375000
91735,C_ID_1670e9e2fa,6.0,44.0,12.0,19.0,30.0,45.0,10.0,7.0,17.0,...,1.0,8.0,57.0,14.0,25.0,27.0,12.0,0,0.110180,23.000000
25347,C_ID_0b358310f5,3.0,270.0,9.0,4.0,12.0,6.0,29.0,17.0,11.0,...,16.0,12.0,8.0,7.0,81.0,8.0,4.0,1,-33.219281,23.083333
10096,C_ID_66b1e819ea,22.0,78.0,14.0,23.0,6.0,29.0,17.0,32.0,4.0,...,27.0,22.0,27.0,32.0,51.0,46.0,39.0,0,1.097145,28.708333
179088,C_ID_ed2b2ab4ea,38.0,54.0,90.0,106.0,19.0,28.0,16.0,44.0,9.0,...,14.0,9.0,28.0,15.0,14.0,12.0,23.0,1,-33.219281,28.958333


In [272]:
rank_data.sort_values('outlier_rank').head(20000).describe()

,yh_target,train_oof_cat_auc_real,train_oof_lgb_auc_real,train_oof_lgb_log_loss_real,train_yeonmin_rank_kfold_stacking_0,train_yeonmin_rank_kfold_stacking_1,train_yeonmin_rank_kfold_stacking_2,train_yeonmin_rank_kfold_stacking_3,train_yeonmin_rank_kfold_stacking_4,train_yeonmin_rank_kfold_stacking2_0,...,train_yeonmin_rank_sfold_stacking_3,train_yeonmin_rank_sfold_stacking_4,train_yeonmin_rank_sfold_stacking2_0,train_yeonmin_rank_sfold_stacking2_1,train_yeonmin_rank_sfold_stacking2_2,train_yeonmin_rank_sfold_stacking2_3,train_yeonmin_rank_sfold_stacking2_4,outlier,target,outlier_rank
count,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,...,20000.000000,20000.00000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,10750.639850,11174.16610,10889.460550,10878.561150,10163.368600,10297.248100,10277.862150,10186.954600,10166.180550,10184.642350,...,10189.042250,10193.20960,10195.541450,10322.76935,10324.179300,10175.210250,10173.402600,0.078850,-2.672489,10346.569706
std,7011.535028,7524.01621,7139.503182,7112.669698,6025.787887,6256.428475,6250.060473,6095.838058,6029.534728,6065.173427,...,6094.766141,6076.19514,6084.502641,6299.18546,6334.784619,6071.678553,6049.310536,0.269511,9.192077,5873.794697
min,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.00000,1.000000,1.00000,1.000000,1.000000,1.000000,0.000000,-33.219281,7.041667
25%,5002.750000,5069.75000,5062.750000,5064.750000,5004.750000,5003.750000,5000.750000,5000.750000,5004.750000,5004.750000,...,5000.750000,5003.75000,5005.750000,5003.75000,5000.750000,5000.750000,5004.750000,0.000000,-1.647408,5342.781250
50%,10057.500000,10269.50000,10207.500000,10208.500000,10020.500000,10019.500000,10002.500000,10000.500000,10021.500000,10018.500000,...,10000.500000,10015.50000,10016.500000,10035.50000,10001.500000,10000.500000,10013.500000,0.000000,-0.176361,10294.854167
75%,15572.500000,16028.50000,15650.250000,15654.250000,15081.250000,15154.250000,15077.250000,15032.250000,15083.250000,15095.500000,...,15040.250000,15091.25000,15097.250000,15170.25000,15107.250000,15031.250000,15074.250000,0.000000,0.959019,15358.156250
max,59865.000000,95809.00000,60458.000000,52082.000000,29495.000000,43032.000000,35132.000000,32807.000000,29302.000000,33803.000000,...,32753.000000,32786.00000,32369.000000,46106.00000,38558.000000,28854.000000,32215.000000,1.000000,15.994455,20639.041667


In [261]:
rank_data.sort_values('outlier_rank').to_csv('rank_median_result.csv',index=False)

In [258]:
rank_data.sort_values('outlier_rank').to_csv('rank_mean_train_result.csv',index=False)

In [92]:
top30_outlier = data.sort_values('outlier_rank').head(30)

In [94]:
top30_outlier.loc[top30_outlier['outlier']==0]

,card_id,target_x,target_y,target_x,target_y,target_x,target_y,predictions_x,predictions_y,predictions_x,predictions_y,0,1,2,3,4,outlier_rank,outlier
91735,C_ID_1670e9e2fa,0.527142,0.527142,0.573086,0.573086,0.589661,0.589661,0.372820,0.513348,0.410636,0.405710,0.493810,0.491698,0.577700,0.565600,0.509425,16.225806,0
157659,C_ID_f7a43b55d0,0.523794,0.523794,0.499536,0.499536,0.541066,0.541066,0.428438,0.462027,0.540625,0.552477,0.507666,0.449075,0.465965,0.483682,0.509633,18.806452,0
21057,C_ID_26e9fcf59c,0.446181,0.446181,0.469304,0.469304,0.520454,0.520454,0.394381,0.691220,0.621308,0.617375,0.603936,0.620223,0.633646,0.673663,0.454215,21.387097,0
15933,C_ID_b7d3533686,0.460587,0.460587,0.474913,0.474913,0.531092,0.531092,0.452363,0.484635,0.473286,0.473275,0.454160,0.489217,0.573211,0.528104,0.460771,23.838710,0
154824,C_ID_c45fd0c84b,0.459751,0.459751,0.462705,0.462705,0.510522,0.510522,0.338926,0.461553,0.405187,0.405177,0.478940,0.501021,0.510070,0.494275,0.483524,37.548387,0
164877,C_ID_f1d256250e,0.470269,0.470269,0.438461,0.438461,0.460694,0.460694,0.360822,0.483522,0.378280,0.379643,0.541196,0.570610,0.443988,0.517180,0.541127,43.064516,0
10096,C_ID_66b1e819ea,0.478626,0.478626,0.484289,0.484289,0.501458,0.501458,0.341446,0.499166,0.321467,0.321209,0.583267,0.515236,0.549994,0.512768,0.587605,45.483871,0
5868,C_ID_18c1e4935e,0.492678,0.492678,0.515015,0.515015,0.429669,0.429669,0.278858,0.478303,0.493942,0.532079,0.517419,0.413915,0.513759,0.489022,0.510380,45.548387,0
94554,C_ID_5ddb217d93,0.444961,0.444961,0.507776,0.507776,0.457204,0.457204,0.284634,0.504264,0.496315,0.523863,0.489622,0.385306,0.526259,0.549687,0.490094,46.387097,0
59479,C_ID_e4208298d5,0.430266,0.430266,0.514592,0.514592,0.423430,0.423430,0.381933,0.479142,0.383899,0.383640,0.447109,0.448835,0.447784,0.518047,0.386910,53.193548,0


In [82]:
data.loc[data['card_id']=='C_ID_1670e9e2fa'].values

array([['C_ID_1670e9e2fa', 0.5271420617939009, 0.5271420617939009,
        0.5730861019653463, 0.5730861019653463, 0.5896610193462387,
        0.5896610193462387, 0.5133483140251498, 0.4938095163796869,
        0.491698067064644, 0.5776995690209878, 0.5656003355425236,
        0.5094250458719055, 0.5358396960557096, 0.5429082516139805,
        0.6347587655547916, 0.7020363653394288, 0.5933376333771243, 6.0,
        0]], dtype=object)

test oof

In [282]:
test_data.columns 

Index(['card_id', 'yh_target', 'train_oof_cat_auc_real',
       'train_oof_lgb_auc_real', 'train_oof_lgb_log_loss_real',
       'train_yeonmin_rank_kfold_stacking_0',
       'train_yeonmin_rank_kfold_stacking_1',
       'train_yeonmin_rank_kfold_stacking_2',
       'train_yeonmin_rank_kfold_stacking_3',
       'train_yeonmin_rank_kfold_stacking_4',
       'train_yeonmin_rank_kfold_stacking2_0',
       'train_yeonmin_rank_kfold_stacking2_1',
       'train_yeonmin_rank_kfold_stacking2_2',
       'train_yeonmin_rank_kfold_stacking2_3',
       'train_yeonmin_rank_kfold_stacking2_4',
       'train_yeonmin_rank_sfold_stacking_0',
       'train_yeonmin_rank_sfold_stacking_1',
       'train_yeonmin_rank_sfold_stacking_2',
       'train_yeonmin_rank_sfold_stacking_3',
       'train_yeonmin_rank_sfold_stacking_4',
       'train_yeonmin_rank_sfold_stacking2_0',
       'train_yeonmin_rank_sfold_stacking2_1',
       'train_yeonmin_rank_sfold_stacking2_2',
       'train_yeonmin_rank_sfold_stacking2_

In [294]:
test_data = pd.DataFrame()
test_data = merge_all_result(test_outlier_path,'card_id',test_data)
test_data.columns = ['card_id', 'yh_target', 'train_oof_cat_auc_real','train_oof_lgb_auc_real', 'train_oof_lgb_log_loss_real',
       'train_yeonmin_rank_kfold_stacking_0', 'train_yeonmin_rank_kfold_stacking_1',
       'train_yeonmin_rank_kfold_stacking_2', 'train_yeonmin_rank_kfold_stacking_3', 'train_yeonmin_rank_kfold_stacking_4',
       'train_yeonmin_rank_kfold_stacking2_0', 'train_yeonmin_rank_kfold_stacking2_1',
       'train_yeonmin_rank_kfold_stacking2_2', 'train_yeonmin_rank_kfold_stacking2_3', 'train_yeonmin_rank_kfold_stacking2_4',
       'train_yeonmin_rank_sfold_stacking_0', 'train_yeonmin_rank_sfold_stacking_1', 'train_yeonmin_rank_sfold_stacking_2', 
       'train_yeonmin_rank_sfold_stacking_3', 'train_yeonmin_rank_sfold_stacking_4', 
       'train_yeonmin_rank_sfold_stacking2_0', 'train_yeonmin_rank_sfold_stacking2_1', 'train_yeonmin_rank_sfold_stacking2_2', 
       'train_yeonmin_rank_sfold_stacking2_3', 'train_yeonmin_rank_sfold_stacking2_4']

test_oof_augmented_2500_auc_0.9057314966569757_logloss__function log_loss at 0x7f9ab3285bf8__v1.csv
empty
test_oof_cat_auc_real.csv
in
test_oof_lgb_auc_real.csv
in
test_oof_lgb_log_loss_real.csv
in
test_yeonmin_rank_kfold_stacking.csv
in
test_yeonmin_rank_kfold_stacking2.csv
in
test_yeonmin_rank_sfold_stacking.csv
in
test_yeonmin_rank_sfold_stacking2.csv
in


In [286]:
for col in col_base_ratio_dict:
    ratio = col_base_ratio_dict[col]
    test_data.loc[test_data[col]>=ratio, col] = 1
    test_data.loc[test_data[col]!=1, col] = 0
    print(col)

yh_target
train_oof_cat_auc_real
train_oof_lgb_auc_real
train_oof_lgb_log_loss_real
train_yeonmin_rank_kfold_stacking_0
train_yeonmin_rank_kfold_stacking_1
train_yeonmin_rank_kfold_stacking_2
train_yeonmin_rank_kfold_stacking_3
train_yeonmin_rank_kfold_stacking_4
train_yeonmin_rank_kfold_stacking2_0
train_yeonmin_rank_kfold_stacking2_1
train_yeonmin_rank_kfold_stacking2_2
train_yeonmin_rank_kfold_stacking2_3
train_yeonmin_rank_kfold_stacking2_4
train_yeonmin_rank_sfold_stacking_0
train_yeonmin_rank_sfold_stacking_1
train_yeonmin_rank_sfold_stacking_2
train_yeonmin_rank_sfold_stacking_3
train_yeonmin_rank_sfold_stacking_4
train_yeonmin_rank_sfold_stacking2_0
train_yeonmin_rank_sfold_stacking2_1
train_yeonmin_rank_sfold_stacking2_2
train_yeonmin_rank_sfold_stacking2_3
train_yeonmin_rank_sfold_stacking2_4


In [300]:
rank_test_data = test_data[col_list].rank(ascending=False)
rank_test_data = pd.concat([test_data[['card_id']],rank_test_data],axis=1)
rank_test_data['outlier_rank'] = rank_test_data[col_list].median(axis=1)

In [303]:
rank_test_data[['card_id','outlier_rank']].to_csv('top_rank_median_outlier.csv',index=False)

In [290]:
test_data['voting'] = test_data[col_list].mean(axis=1)

In [293]:
test_data[['card_id','voting']].to_csv('top_voting_outlier.csv',index=False)